In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import json
import math
import os
import cv2
from tqdm import tqdm

In [ ]:
train_labels = []
train_images_path = []
for dirname, _, filenames in os.walk('../input/super-ai-engineer-2021-font-recognition/train/set1'):
    for filename in filenames:
        if filename.endswith('.json'):
            path = os.path.join(dirname, filename)
            f = open(path)
            labels = json.load(f)
            train_labels.append(labels)
        if filename.endswith('.png'):
            train_images_path.append(os.path.join(dirname, filename))
print(len(train_labels))
print(len(train_images_path))

In [ ]:
valid_labels = []
valid_images_path = []
for dirname, _, filenames in os.walk('../input/super-ai-engineer-2021-font-recognition/train/set2'):
    for filename in filenames:
        if filename.endswith('.json'):
            path = os.path.join(dirname, filename)
            f = open(path)
            labels = json.load(f)
            valid_labels.append(labels)
        if filename.endswith('.png'):
            valid_images_path.append(os.path.join(dirname, filename))
print(len(valid_labels))
print(len(valid_images_path))

In [ ]:
valid_labels[0][0]

In [ ]:
def crop_img(image_path,x,y,w,h,i,j,is_train):
    try:
        image = cv2.imread(image_path)
        crop_image =  image[math.floor(y):math.ceil(y+h),math.floor(x):math.ceil(x+w)]
        path = 'train_images/image'+str(i)+'_'+str(j)+'.png' if is_train else 'test_images/image'+str(i)+'_'+str(j)+'.png'
        cv2.imwrite(path,crop_image)
    except:
        return 'empty image'
    return path

def process_data(image_path_list,json_labels_list,is_train):
    if is_train:
        os.mkdir('train_images')
    else:
        os.mkdir('test_images')
        
    max_w = 0
    max_h = 0
    df = pd.DataFrame()
    images = []
    texts = []
    fontSizes = []
    fontFamilys = []
    fontStyles = []
#         fontVarients = []
    fontWeights = []
    for i,(json_labels,image_path) in tqdm(enumerate(zip(json_labels_list,image_path_list))):
        for j,json_label in enumerate(json_labels):
            x = json_label['rect']['x']
            y = json_label['rect']['y']
            w = json_label['rect']['width']
            h = json_label['rect']['height']
            max_w = w if w>max_w else max_w
            max_h = h if h>max_h else max_h
            image = crop_img(image_path,x,y,w,h,i,j,is_train)
            if image != 'empty image':
                images.append(image)
                texts.append(json_label['text'])
                fontSizes.append(json_label['style']['fontSize'])
                fontFamilys.append(json_label['style']['fontFamily'])
                fontStyles.append(json_label['style']['fontStyle'])
            #             fontVarients.append(json_label['style']['fontVarient'])
                fontWeights.append(json_label['style']['fontWeight'])
#         temp_df['images'] = images
#         temp_df['text'] = texts
#         temp_df['fontSize'] = fontSizes
#         temp_df['fontFamily'] = fontFamilys
#         temp_df['fontStyle'] = fontStyles
# #         temp_df['fontVarient'] = fontVarients
#         temp_df['fontWeight'] = fontWeights
#         df.append(temp_df)
    df['images'] = images
    df['text'] = texts
    df['fontSize'] = fontSizes
    df['fontFamily'] = fontFamilys
    df['fontStyle'] = fontStyles
#         temp_df['fontVarient'] = fontVarients
    df['fontWeight'] = fontWeights
    return df, max_w, max_h
        
        

In [ ]:
train_df,max_w,max_h = process_data(train_images_path,train_labels,True)
max_wh = pd.DataFrame([[max_w,max_h]],columns=['max_w','max_h'])
max_wh.to_csv('train_max_wh.csv',index=False)
train_df.to_csv('train_data.csv',index=False)


In [ ]:
train_df

In [ ]:
test_df,max_w,max_h = process_data(train_images_path,train_labels,False)
max_wh = pd.DataFrame([[max_w,max_h]],columns=['max_w','max_h'])
max_wh.to_csv('test_max_wh.csv',index=False)
train_df.to_csv('test_data.csv',index=False)

In [ ]:
test_df